# Experiments

### Setup

In [1]:
# You can set them inline
import os

In [2]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../../.env", override=True)

True

Here is the RAG Application that we've been working with throughout this course

In [12]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-3.5-turbo"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [13]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "RAG Application Golden Dataset"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

View the evaluation results for experiment: 'gpt-4o-1b842da1' at:
https://smith.langchain.com/o/b596d003-75cc-4859-ab5f-9c3358306b32/datasets/b915d9ee-5e1f-40cd-9481-edca3b51d3be/compare?selectedSessions=a73bb488-bf44-4085-b17e-5bc37a0a705d




0it [00:00, ?it/s]/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
1it [00:01,  1.91s/it]/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeW

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith using LangChain...,1,1.906404,3581a3be-eb8c-4433-938c-44fb6fb1fcd3,f26257ea-c99b-4974-afd0-3fbbd8f6646f
1,Can LangSmith be used for finetuning and model...,"LangSmith is primarily focused on monitoring, ...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.942355,3d109c68-2d9d-4f2f-9423-9411765e1a90,494e2f72-5d44-40ec-8da6-8d6ee5221095
2,Does LangSmith support offline evaluation?,LangSmith does not currently support offline e...,None,"Yes, LangSmith supports offline evaluation thr...",0,2.254435,570c65da-4c9c-46a3-a940-ba60ade0ea95,2d9134d9-13a2-4d70-bf8d-10a494634613
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,2.376836,629c77b5-8932-4d3a-8116-08aa9e14264d,11ebb0d7-62af-4ee3-8373-606e5b2e870f
4,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,2.064654,6a09ed1b-6080-4a23-ab01-401e8695bfe2,f9cfd538-bbac-4157-b88d-6fff861de6e5
5,What testing capabilities does LangSmith have?,LangSmith in LangChain offers testing capabili...,None,LangSmith offers capabilities for creating dat...,1,1.551240,702a5561-a963-413d-a74d-6eeb0f9ff85f,edc2301d-ee51-41e9-bfae-289828617873
6,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents,...",None,"Yes, LangSmith can be used to evaluate agents....",0,1.714372,8574e306-ee97-409e-b32f-f3785d93210f,c7bd89da-c324-4bc4-9479-6521773f0ff8
7,What is LangSmith used for in three sentences?,LangSmith is a platform used for building prod...,None,LangSmith is a platform designed for the devel...,1,1.803292,8b1aa813-0c6b-4d66-a45a-56724c9daab2,7984f1eb-a797-48b9-bfe3-d5a977cf4374
8,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.864926,9c5cc3f5-2868-4636-96e7-1179db005389,b67e0e1e-2dbd-42a7-8376-727219ed7e66
9,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.266596,a01d92b2-8ba2-4d53-8333-76dd8e81e843,5a6e2457-64b0-4919-81fe-ef7baea1f176


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [14]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-f9562786' at:
https://smith.langchain.com/o/b596d003-75cc-4859-ab5f-9c3358306b32/datasets/b915d9ee-5e1f-40cd-9481-edca3b51d3be/compare?selectedSessions=015fa27e-f854-4aeb-b5b9-4aaeef00a9c3




0it [00:00, ?it/s]/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
1it [00:02,  2.67s/it]/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeW

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith using LangChain...,1,2.663210,3581a3be-eb8c-4433-938c-44fb6fb1fcd3,3330b017-4aa5-4977-a1ac-52ba638a0d18
1,Can LangSmith be used for finetuning and model...,LangSmith is primarily focused on observabilit...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.515153,3d109c68-2d9d-4f2f-9423-9411765e1a90,07965d84-56d9-4ceb-91a7-9ff276b5cd91
2,Does LangSmith support offline evaluation?,LangSmith primarily supports online evaluation...,None,"Yes, LangSmith supports offline evaluation thr...",1,2.055972,570c65da-4c9c-46a3-a940-ba60ade0ea95,f55b551e-7852-4094-930c-3b92dcb23418
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,0,2.327553,629c77b5-8932-4d3a-8116-08aa9e14264d,57ed7dbb-3771-46c4-a701-1398edf2feb4
4,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,2.588893,6a09ed1b-6080-4a23-ab01-401e8695bfe2,ecb270ca-a4af-4cc2-acab-edb8bc2d88af
5,What testing capabilities does LangSmith have?,LangSmith in LangChain offers the ability to r...,None,LangSmith offers capabilities for creating dat...,1,2.436800,702a5561-a963-413d-a74d-6eeb0f9ff85f,5c2ddd8e-390a-4a5c-a336-ab8bc4bddd48
6,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",1,1.681056,8574e306-ee97-409e-b32f-f3785d93210f,51606588-e746-4010-987a-d9838e57a2b5
7,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.672486,8b1aa813-0c6b-4d66-a45a-56724c9daab2,1cacad8b-6893-49f6-80a6-42440df57d6e
8,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,2.274989,9c5cc3f5-2868-4636-96e7-1179db005389,4482267a-48e7-4540-ae26-0e80bc1ba15d
9,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,0,2.487885,a01d92b2-8ba2-4d53-8333-76dd8e81e843,03b65883-c843-48f9-9885-0b9fb3525112


### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

In [16]:
evaluate(
    target_function,
    data=dataset_name,  # Pass the dataset name directly to use the latest version
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)

View the evaluation results for experiment: 'initial dataset version-140a518d' at:
https://smith.langchain.com/o/b596d003-75cc-4859-ab5f-9c3358306b32/datasets/b915d9ee-5e1f-40cd-9481-edca3b51d3be/compare?selectedSessions=a28c04bb-4ac2-4f89-9e8e-44ba9e7a3c1d




0it [00:00, ?it/s]/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
1it [00:02,  2.96s/it]/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeW

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith using LangChain...,1,2.961468,3581a3be-eb8c-4433-938c-44fb6fb1fcd3,4834f3bd-8b39-4f54-9abf-8adc04fbdc32
1,Can LangSmith be used for finetuning and model...,"LangSmith is primarily focused on monitoring, ...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.952138,3d109c68-2d9d-4f2f-9423-9411765e1a90,19857017-14bb-436d-91c9-4fcc1dd2bd70
2,Does LangSmith support offline evaluation?,LangSmith primarily supports online evaluation...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.638284,570c65da-4c9c-46a3-a940-ba60ade0ea95,e7164353-ade7-4cd3-a574-b139bb283634
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,2.561685,629c77b5-8932-4d3a-8116-08aa9e14264d,2d45d600-e490-4534-bac2-acc1de4cbb7d
4,Does LangSmith support online evaluation?,"LangSmith supports online evaluation, offering...",None,"Yes, LangSmith supports online evaluation as a...",0,1.807753,6a09ed1b-6080-4a23-ab01-401e8695bfe2,3a6cf148-2fdf-4a7b-a7f1-ea360144c238
5,What testing capabilities does LangSmith have?,LangSmith allows users to run multiple experim...,None,LangSmith offers capabilities for creating dat...,1,1.459471,702a5561-a963-413d-a74d-6eeb0f9ff85f,c7943c56-e260-4193-859f-af214db77a18
6,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",0,1.540678,8574e306-ee97-409e-b32f-f3785d93210f,8ecd40b0-c963-428b-adcc-4fcc28890dbd
7,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.596673,8b1aa813-0c6b-4d66-a45a-56724c9daab2,b162fdb3-3f01-4293-8067-e46b246a49fd
8,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.926912,9c5cc3f5-2868-4636-96e7-1179db005389,6dbf139d-30cc-4939-926b-7580a29d8ee3
9,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,0,1.674753,a01d92b2-8ba2-4d53-8333-76dd8e81e843,b8b9f663-1ef2-4d25-ba57-66380fc25240


##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [17]:
evaluate(
    target_function,
    data=dataset_name,  # Pass the dataset name directly to use the latest version
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)

View the evaluation results for experiment: 'initial dataset version-7b80e968' at:
https://smith.langchain.com/o/b596d003-75cc-4859-ab5f-9c3358306b32/datasets/b915d9ee-5e1f-40cd-9481-edca3b51d3be/compare?selectedSessions=0df8e857-f100-4629-b8b2-d2f7cbcc91c1




0it [00:00, ?it/s]/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
1it [00:01,  1.63s/it]/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeW

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith using LangChain...,1,1.629751,3581a3be-eb8c-4433-938c-44fb6fb1fcd3,1eff7127-dc49-4681-baf9-36598d286a37
1,Can LangSmith be used for finetuning and model...,"LangSmith is primarily focused on monitoring, ...",None,"Yes, LangSmith can be used for fine-tuning and...",0,1.727494,3d109c68-2d9d-4f2f-9423-9411765e1a90,8e9620cb-b9b7-4a19-bd77-26ec30af6e98
2,Does LangSmith support offline evaluation?,LangSmith primarily supports online evaluation...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.621533,570c65da-4c9c-46a3-a940-ba60ade0ea95,6d43f273-2f25-4358-85eb-c042b9cb1cb9
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,0,2.358270,629c77b5-8932-4d3a-8116-08aa9e14264d,2a68527a-32f4-4e2e-93f1-f7477b3f59ca
4,Does LangSmith support online evaluation?,"Yes, LangChain's LangSmith supports online eva...",None,"Yes, LangSmith supports online evaluation as a...",0,2.205589,6a09ed1b-6080-4a23-ab01-401e8695bfe2,5b0e65ed-8a35-4b69-8089-345edce6c12f
5,What testing capabilities does LangSmith have?,LangSmith allows users to run multiple experim...,None,LangSmith offers capabilities for creating dat...,1,1.435877,702a5561-a963-413d-a74d-6eeb0f9ff85f,f5a829c3-15d4-4d49-895f-889038bb2ff5
6,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",0,2.519077,8574e306-ee97-409e-b32f-f3785d93210f,eb21ec16-67ef-4ed4-9f5a-073f9eb932f7
7,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.747493,8b1aa813-0c6b-4d66-a45a-56724c9daab2,e6c38620-ae91-4eb1-a9a4-506428869e8a
8,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.695263,9c5cc3f5-2868-4636-96e7-1179db005389,fa92a32b-1106-4a31-a5a9-dcb7ace66336
9,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.938508,a01d92b2-8ba2-4d53-8333-76dd8e81e843,7d7ab37f-1a90-4cbb-9f46-511de7b78be1


##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [18]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "3581a3be-eb8c-4433-938c-44fb6fb1fcd3",
            "3d109c68-2d9d-4f2f-9423-9411765e1a90"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-11125639' at:
https://smith.langchain.com/o/b596d003-75cc-4859-ab5f-9c3358306b32/datasets/b915d9ee-5e1f-40cd-9481-edca3b51d3be/compare?selectedSessions=a190e46e-3e73-42b6-b462-cfce02c8dfbb




0it [00:00, ?it/s]/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
1it [00:01,  1.93s/it]/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeW

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith using LangChain...,1,1.92979,3581a3be-eb8c-4433-938c-44fb6fb1fcd3,df29823d-d8d5-44d9-9e2f-1e7ca2bb02ee
1,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for observabil...,None,"Yes, LangSmith can be used for fine-tuning and...",0,1.61088,3d109c68-2d9d-4f2f-9423-9411765e1a90,b5226efa-7f0f-43b7-8fec-8594738603c5


### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [19]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-99c6ce66' at:
https://smith.langchain.com/o/b596d003-75cc-4859-ab5f-9c3358306b32/datasets/b915d9ee-5e1f-40cd-9481-edca3b51d3be/compare?selectedSessions=04e5628d-19f4-462f-bb3f-ccad78aa9013




0it [00:00, ?it/s]/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
1it [00:02,  2.66s/it]/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeW

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith using LangChain...,0,2.661339,3581a3be-eb8c-4433-938c-44fb6fb1fcd3,82c3ef7b-61d7-4106-b351-8517fc255e16
1,Can LangSmith be used for finetuning and model...,LangSmith can be used for tracing an applicati...,None,"Yes, LangSmith can be used for fine-tuning and...",1,3.687266,3d109c68-2d9d-4f2f-9423-9411765e1a90,f27114e8-ac48-4c35-afcb-4f663b4c21f7
2,Does LangSmith support offline evaluation?,LangSmith primarily focuses on online evaluati...,None,"Yes, LangSmith supports offline evaluation thr...",0,2.888574,570c65da-4c9c-46a3-a940-ba60ade0ea95,e17b9943-d7cf-45ab-b0bc-735ad0e47336
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,2.390692,629c77b5-8932-4d3a-8116-08aa9e14264d,b5c41bbe-4d09-4e73-8e04-22f516bac018
4,Does LangSmith support online evaluation?,LangSmith supports online evaluation through i...,None,"Yes, LangSmith supports online evaluation as a...",0,2.262539,6a09ed1b-6080-4a23-ab01-401e8695bfe2,ab36384e-2314-4c70-ac06-9435cc19c2f1
5,What testing capabilities does LangSmith have?,LangSmith allows running multiple experiments ...,None,LangSmith offers capabilities for creating dat...,1,1.434105,702a5561-a963-413d-a74d-6eeb0f9ff85f,8594de23-1f78-4c63-b716-b6fe7167438a
6,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents,...",None,"Yes, LangSmith can be used to evaluate agents....",0,3.078178,8574e306-ee97-409e-b32f-f3785d93210f,e41bc4fe-96dd-4d2d-88d4-b67956bfd9b2
7,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,2.159645,8b1aa813-0c6b-4d66-a45a-56724c9daab2,88446837-5014-40fa-8fb5-a27e5d2339cf
8,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,3.362805,9c5cc3f5-2868-4636-96e7-1179db005389,1eb766b9-8c78-4a97-8d27-3eadb1df5c6f
9,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.826855,a01d92b2-8ba2-4d53-8333-76dd8e81e843,ef7df0fb-690b-44b2-a67d-f4c3324cc837


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [20]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-d3b18655' at:
https://smith.langchain.com/o/b596d003-75cc-4859-ab5f-9c3358306b32/datasets/b915d9ee-5e1f-40cd-9481-edca3b51d3be/compare?selectedSessions=1faf4e08-c371-4421-a11c-1942df83f6e6




0it [00:00, ?it/s]/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value 

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support offline evaluation?,LangSmith supports online evaluations for real...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.506998,570c65da-4c9c-46a3-a940-ba60ade0ea95,df3ae476-ee96-49ec-bf24-1484676833ac
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith using LangChain...,1,1.604791,3581a3be-eb8c-4433-938c-44fb6fb1fcd3,865a3c8d-e781-4e8a-b6a7-15750d717e71
2,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,1.774168,629c77b5-8932-4d3a-8116-08aa9e14264d,0ccda044-b3b8-46fd-b6ab-697e3e0b0efb
3,Does LangSmith support online evaluation?,LangSmith does support online evaluation throu...,None,"Yes, LangSmith supports online evaluation as a...",0,2.064098,6a09ed1b-6080-4a23-ab01-401e8695bfe2,98e1eb8a-1cca-4a85-b8db-19b9af24fc26
4,What testing capabilities does LangSmith have?,LangSmith allows running multiple experiments ...,None,LangSmith offers capabilities for creating dat...,1,1.514326,702a5561-a963-413d-a74d-6eeb0f9ff85f,5ad650e4-9f56-46b0-ae73-ce04da392e04
5,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.569926,8b1aa813-0c6b-4d66-a45a-56724c9daab2,fa38aac1-6bcc-4965-abb8-a0c1eac9bb50
6,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",1,3.988240,8574e306-ee97-409e-b32f-f3785d93210f,22b14d4d-3a86-491a-97cd-eaff25a4fd99
7,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ca...",None,To trace with the @traceable decorator in Pyth...,1,2.108683,9c5cc3f5-2868-4636-96e7-1179db005389,41a42df6-a7ba-4baf-8d8b-879a99142573
8,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.618716,a01d92b2-8ba2-4d53-8333-76dd8e81e843,687dc19d-716d-4e0e-83f6-7bf03e7ae352
9,How do I pass metadata in with @traceable?,"To pass metadata in with `@traceable`, you can...",None,You can pass metadata with the @traceable deco...,0,1.996838,c4fadbef-f20d-42cd-a5cc-7fa848f1e8a8,7a475480-576c-4d46-a4d7-a9b6e5da1d56


##### Metadata 

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [21]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-c242c2f4' at:
https://smith.langchain.com/o/b596d003-75cc-4859-ab5f-9c3358306b32/datasets/b915d9ee-5e1f-40cd-9481-edca3b51d3be/compare?selectedSessions=5fb245e2-58b8-487b-969f-9eba19424488




0it [00:00, ?it/s]/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
1it [00:01,  1.67s/it]/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeW

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith using LangChain...,1,1.667358,3581a3be-eb8c-4433-938c-44fb6fb1fcd3,4913fec9-99b2-4958-93ea-d12773f448d3
1,Can LangSmith be used for finetuning and model...,LangSmith is primarily focused on observabilit...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.441443,3d109c68-2d9d-4f2f-9423-9411765e1a90,903e2e01-88ce-4965-ab05-175324b73334
2,Does LangSmith support offline evaluation?,LangSmith primarily supports online evaluation...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.573078,570c65da-4c9c-46a3-a940-ba60ade0ea95,6fe8095b-6f04-4c7e-a21e-4a9c8f4fe9fb
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,1.634843,629c77b5-8932-4d3a-8116-08aa9e14264d,07e98ca2-3201-4a7a-ae59-cbd866a980d1
4,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,1.844937,6a09ed1b-6080-4a23-ab01-401e8695bfe2,5c4a68b8-44da-44c3-9530-32ed2e9fa4f6
5,What testing capabilities does LangSmith have?,LangSmith in LangChain offers the capability t...,None,LangSmith offers capabilities for creating dat...,1,2.031456,702a5561-a963-413d-a74d-6eeb0f9ff85f,4623dcef-e6b6-4683-be1f-80dba40f4feb
6,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",0,1.568399,8574e306-ee97-409e-b32f-f3785d93210f,e4a86d73-30f9-48a5-8ee7-b384d679ea28
7,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.715859,8b1aa813-0c6b-4d66-a45a-56724c9daab2,6887a6c5-38d6-4111-aa9a-7ba0788d81c3
8,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.706699,9c5cc3f5-2868-4636-96e7-1179db005389,2d4bb6c5-0ae7-4195-aaa9-77f84c3c5c17
9,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.281510,a01d92b2-8ba2-4d53-8333-76dd8e81e843,d7c5a576-7c4f-4977-8ee5-bc300d7af8bc
